In [ ]:
from ray.tune.integration.mlflow import mlflow_mixin 
import mlflow
import flash

In [ ]:
@mlflow_mixin
def train_dl_model():
    mlflow.pytorch.autolog()
    trainer = flash.Trainer(
    )
    trainer.finetune()